# Point Spectrum

Given a matrix $B$, let $\chi_B$ denote the characteristic polynomial.

For a point $E$ to lay in the point spectrum of $H$ (cf. Proposition 3.8) we need that
$$
\chi_{H_{0..K-2}}(E) = 0
$$
and
$$
-1 < \chi_{H_{0..K-1}}(E) < 1.
$$

According to Remark 3.9 these values correspond to the entries $M_{2,1}(E)$ and $M_{1,1}(E)$ of the monodromy matrix.

In [2]:
# Run this cell once at startup

l = var('l')
E = 0

def mon(v):
#
# Calculate the general monodromy matrix with scaling l #
# and energy E
#
# Example:
# v = [1,1,0]; mon(v).expand()
# > [E^2 - E*l - 1            -E]
#   [        E - l            -1]
#
##########################
    M = identity_matrix(2)
    for vv in v:
        M = matrix(2,2,[E - l*vv, -1, 1, 0])*M
    return M

def lim_pots(v):
#
# determine all unique limit potentials of a given
# periodic potential v by considering all shifts and flips
#
##########################
    v_rev = deepcopy(v)
    lim = shift_pots(v)
    v_rev.reverse()
    lim_rev = shift_pots(v_rev)
    for pot in lim_rev:
        if pot not in lim:
            lim.append(pot)
    return lim
    
def shift_pots(v):
#
# determine all unique shifts of a given periodic potential v
#
##########################
    shift = []
    for c in range(len(v)):
        if v not in shift:
            shift.append(v)
        v = right_shift(v)
    return shift
    
def right_shift(a):
    return [a[-1] , *a[:-1]]


In [96]:
# this cell contains basic routines that need to be loaded 
# before data generation can start

def analyse_v(v):
##########################
# calculate monodromy, zeros and check trace. 
# return a dictionary containing symbolic elements
# 
# This routine analyses the scaling l = ll.
#
# example:
# v = [1,1,0]; analyse_v(v,0.1)
##########################
    data = {}
#monodromy
    M = mon(v).expand() 
#zeros
    sols = solve(M[1][0] == 0, l, to_poly_solve=True)
    sol = list(map(lambda s: real(s.rhs().n()), sols))
    sol.sort()
    data['sol'] = sol
    data['cond'] = list(map(lambda s: real(M[0,0].subs(l=s).n()), sol))
    return data
   
def build_dict(v,nsteps,start,end):
    v_dict = {}
    for s in range(nsteps + 1):
        ll = start + end * s/nsteps
        v_dict[ll] =  analyse_v(v, ll)
    return v_dict
        
def hagger_filter(H, eps=0.01):
    isev = {}
    traj = []
    for ev in range(len(H['sol'])):
        if (ev == 0):
            continue
        isev[ev] = False
        if(abs(H['cond'][ev].n()) < 1 - eps):
            isev[ev] = True
            traj.append([H['sol'][ev],H['cond'][ev]])
    return traj

import csv
def export_traj(v,H):
    for ev in H:
        
        prefix = "data/"
        outname = prefix + str(list(v)).replace(", ", "").replace("[", "").replace("]", "") + "_" +  str(ev) + ".csv"
        f = open(outname, 'w')
        writefile = csv.writer(f)
        for line in H[ev]:
            writefile.writerow(line)
        f.close()
        
def is_same_shift(list_b, list_a):
    if(len(list_a) != len(list_b)):
        return False
    
    for j in range(len(list_a)):
        found = False
        for k in range(len(list_b)):
            if(list_a[j] == list_b[k]):
                found = True
                break
        if found == False:
            return False
    
    return True 

In [104]:
v = [1,0,1,1, 0,1,1,1,0]
v_rev = deepcopy(v)
v_rev.reverse()
v_list = shift_pots(v)
v_rev_list = shift_pots(v_rev)


H = {}
nsteps =150
start = 0
end = 1.5
evs = {}

if (is_same_shift(v_list, v_rev_list)):
    print("Both shifts are equal")
    # print(v_list)
    # print("----")
    # print(v_rev_list)
else:
    result_list = []
    for vv in v_list:
        # print("Checking potential:", vv)
        H[tuple(vv)] = analyse_v(vv)
        evs[tuple(vv)] = hagger_filter(H[tuple(vv)])
        for k in range(len(evs[tuple(vv)])):
            ev = evs[tuple(vv)][k]
            if(len(ev) < 2):
                continue
            result_list.append({'potential':vv, 'lambda':ev[0], 'cond': ev[1] })

    result_list_rev = []
    for vv in v_rev_list:
        # print("Checking potential:", vv)
        H[tuple(vv)] = analyse_v(vv)
        evs[tuple(vv)] = hagger_filter(H[tuple(vv)])
        for k in range(len(evs[tuple(vv)])):
            ev = evs[tuple(vv)][k]
            if(len(ev) < 2):
                continue
            result_list_rev.append({'potential':vv, 'lambda':ev[0], 'cond': ev[1] })

for res in result_list:        
    print(res)
    print(mon(res['potential']).subs(l=1/sqrt(2)))
print("------ Reversed -------")  
for res in result_list_rev:        
    print(res)
    print(mon(res['potential']).subs(l=1/sqrt(2)))

{'potential': [1, 0, 1, 1, 0, 1, 1, 1, 0], 'lambda': -0.396143495696631, 'cond': 0.939763816175616}
[-1/2*sqrt(2)            0]
[        -3/2     -sqrt(2)]
{'potential': [1, 0, 1, 1, 0, 1, 1, 1, 0], 'lambda': 0.396143495696631, 'cond': -0.939763816175616}
[-1/2*sqrt(2)            0]
[        -3/2     -sqrt(2)]
{'potential': [1, 1, 0, 1, 0, 1, 1, 0, 1], 'lambda': 0.577350269189626, 'cond': -0.577350269189626}
[           0            2]
[        -1/2 -3/2*sqrt(2)]
{'potential': [1, 1, 1, 0, 1, 0, 1, 1, 0], 'lambda': 1.19760533812716, 'cond': 0.362605720002692}
[-5/4*sqrt(2)          1/2]
[        -3/4 -1/4*sqrt(2)]
{'potential': [0, 1, 1, 1, 0, 1, 0, 1, 1], 'lambda': 0.577350269189626, 'cond': -0.577350269189626}
[-1/4*sqrt(2)          3/4]
[        -1/2 -5/4*sqrt(2)]
{'potential': [1, 1, 0, 1, 1, 1, 0, 1, 0], 'lambda': -0.437016024448821, 'cond': 0.707106781186547}
[-1/2*sqrt(2)            0]
[          -1     -sqrt(2)]
{'potential': [1, 1, 0, 1, 1, 1, 0, 1, 0], 'lambda': 0.43701602444

In [99]:
0.707106781186548^2

0.500000000000001

In [ ]:
def is_eisenstein(pol):
    params = pol.list()
    for a in params:
        factorlist.append([list(factor(a))])
        
    an_factors = factorlist.pop(0)
    an_primes = [f[0] for f in an_factors]
    
    for p in Primes():
        if p >= min(a):
            break
        
        found = True
        
       # p must not be factor of a_n        
        if p in an_primes:
            found = false
            continue

        # p must be factor of all other a_k
        for facs in factorlist:
            primefactors = [f[0] for f in facs]
            if p not in primefactors:
                found = False
                break
         
        # p^2 must not divide a_0
        multiplicity_a0 = [f[1] for f in factorlist[-1]] 
        if multiplicity_a0 > 1:
            found = False
            continue
        
        if found:
            return True
    
    return False